In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os
import glob

# --- CONFIGURATION ---
MY_LAST_NAME = "Bocala" 

# --- FOLDER NAMES (UPDATED) ---
MC_FOLDER = './muffin_chihuahua'       # For Task 1 & 2
CUSTOM_FOLDER = './pizza_notpizza'     # For Task 3 & 4

IMG_SIZE = (150, 150)
BATCH_SIZE = 20

# --- DATA GENERATORS ---
datagen = ImageDataGenerator(rescale=1./255)

print(f"Loading Task 1 & 2 Data from: {MC_FOLDER}")
# Check if folder exists to prevent crash
if not os.path.exists(MC_FOLDER):
    print(f"ERROR: Folder '{MC_FOLDER}' not found. Please check your directory.")
else:
    train_mc = datagen.flow_from_directory(
        os.path.join(MC_FOLDER, 'train'),
        target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

    test_mc = datagen.flow_from_directory(
        os.path.join(MC_FOLDER, 'test'),
        target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

print(f"Loading Task 3 & 4 Data from: {CUSTOM_FOLDER}")
if not os.path.exists(CUSTOM_FOLDER):
    print(f"ERROR: Folder '{CUSTOM_FOLDER}' not found. Please check your directory.")
else:
    train_custom = datagen.flow_from_directory(
        os.path.join(CUSTOM_FOLDER, 'train'),
        target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

    test_custom = datagen.flow_from_directory(
        os.path.join(CUSTOM_FOLDER, 'test'),
        target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

Loading Task 1 & 2 Data from: ./muffin_chihuahua
Found 4733 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Loading Task 3 & 4 Data from: ./pizza_notpizza
Found 1966 images belonging to 2 classes.
Found 1966 images belonging to 2 classes.


In [17]:
print("\n--- STARTING TASK 2: IMPROVED CNN ---")

# 1. Define Model with Improvements
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    
    # --- REQUIRED IMPROVEMENTS ---
    layers.Dropout(0.5), 
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 2. Train on Muffin/Chihuahua
history = model.fit(train_mc, epochs=5, validation_data=test_mc)

# 3. Answer Question 2a (Accuracy)
acc = history.history['val_accuracy'][-1]
print(f"\n[ANSWER Q2a] Final Accuracy: {acc*100:.2f}%")

# 4. Answer Question 2b (Prediction & Confidence)
print("\n[ANSWER Q2b] Prediction Check:")
try:
    # Pick first image in test folder
    sample_path = glob.glob(f'{MC_FOLDER}/test/*/*.jpg')[0]
    
    img = load_img(sample_path, target_size=IMG_SIZE)
    x = img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    
    pred = model.predict(x)[0][0]
    lbl = "Class 1" if pred > 0.5 else "Class 0"
    conf = pred if pred > 0.5 else 1-pred
    
    print(f"Image: {sample_path}")
    print(f"Prediction: {lbl} | Confidence: {conf:.2%}")
except:
    print("Could not find a test image to predict.")

# 5. Answer Question 2c (Save Model)
model.save('exercise_6_trained_model_improved.h5')
print(">>> SAVED: exercise_6_trained_model_improved.h5")


--- STARTING TASK 2: IMPROVED CNN ---
Epoch 1/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 258s 1s/step - accuracy: 0.7148 - loss: 0.8068 - val_accuracy: 0.8480 - val_loss: 0.4837
Epoch 2/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.8358 - loss: 0.4532 - val_accuracy: 0.8843 - val_loss: 0.3767
Epoch 3/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 203s 856ms/step - accuracy: 0.8519 - loss: 0.4124 - val_accuracy: 0.8801 - val_loss: 0.3738
Epoch 4/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 198s 835ms/step - accuracy: 0.8633 - loss: 0.3772 - val_accuracy: 0.8750 - val_loss: 0.3829
Epoch 5/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 194s 819ms/step - accuracy: 0.8749 - loss: 0.3653 - val_accuracy: 0.8885 - val_loss: 0.3482

[ANSWER Q2a] Final Accuracy: 88.85%

[ANSWER Q2b] Prediction Check:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step


Image: ./muffin_chihuahua/test\chihuahua\img_0_1071.jpg
Prediction: Class 0 | Confidence: 92.66%
>>> SAVED: exercise_6_trained_model_improved.h5


In [19]:
# ==========================================
# TASK 3: CUSTOM DATASET (Clean Version)
# ==========================================
import os
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. CONFIGURATION
MY_LAST_NAME = "Bocala" 
# Use the fixed path. 
# If you moved the folders, change this to './pizza_notpizza'
# If it's still nested, keep './pizza_notpizza/pizza_notpizza'
CUSTOM_FOLDER = './pizza_notpizza' 

IMG_SIZE = (150, 150)
BATCH_SIZE = 20

# 2. LOAD DATA (Creates 'train_custom' so the code works)
print(f"Loading data from: {CUSTOM_FOLDER}")
datagen = ImageDataGenerator(rescale=1./255)

if os.path.exists(CUSTOM_FOLDER):
    train_custom = datagen.flow_from_directory(
        os.path.join(CUSTOM_FOLDER, 'train'),
        target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

    test_custom = datagen.flow_from_directory(
        os.path.join(CUSTOM_FOLDER, 'test'),
        target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
else:
    print("❌ ERROR: Folder not found. Check the path.")

# 3. TRAIN MODEL (Original Task 3 Logic)
print("Starting Training...")

# Clone the Task 2 model structure
model_custom = models.clone_model(model)
model_custom.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model_custom.fit(train_custom, epochs=5, validation_data=test_custom)

# 4. SAVE
filename = f'exercise_6_custom_{MY_LAST_NAME}.h5'
model_custom.save(filename)
print(f">>> SAVED SUCCESSFULLY: {filename}")

Loading data from: ./pizza_notpizza
Found 1966 images belonging to 2 classes.
Found 1966 images belonging to 2 classes.
Starting Training...
Epoch 1/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 98s 960ms/step - accuracy: 0.6104 - loss: 1.1129 - val_accuracy: 0.6628 - val_loss: 0.7834
Epoch 2/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 90s 914ms/step - accuracy: 0.7045 - loss: 0.6806 - val_accuracy: 0.7731 - val_loss: 0.5890
Epoch 3/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 90s 909ms/step - accuracy: 0.7228 - loss: 0.6057 - val_accuracy: 0.7798 - val_loss: 0.5192
Epoch 4/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 90s 906ms/step - accuracy: 0.7675 - loss: 0.5354 - val_accuracy: 0.7976 - val_loss: 0.5098
Epoch 5/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 91s 916ms/step - accuracy: 0.7813 - loss: 0.5287 - val_accuracy: 0.8225 - val_loss: 0.4619


>>> SAVED SUCCESSFULLY: exercise_6_custom_Bocala.h5


In [20]:
print("\n--- STARTING TASK 4: TRANSFER LEARNING (VGG16) ---")

# 1. Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False 

# 2. Create Transfer Model
transfer_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

transfer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Train on PIZZA/NOTPIZZA
transfer_model.fit(train_custom, epochs=5, validation_data=test_custom)

print(">>> TASK 4 COMPLETE. ALL EXERCISES FINISHED.")


--- STARTING TASK 4: TRANSFER LEARNING (VGG16) ---
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 41s 1us/step
Epoch 1/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.8057 - loss: 0.4652 - val_accuracy: 0.9064 - val_loss: 0.2674
Epoch 2/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.8830 - loss: 0.2902 - val_accuracy: 0.9430 - val_loss: 0.1818
Epoch 3/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 259s 3s/step - accuracy: 0.9095 - loss: 0.2243 - val_accuracy: 0.9573 - val_loss: 0.1401
Epoch 4/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.9334 - loss: 0.1785 - val_accuracy: 0.9695 - val_loss: 0.1006
Epoch 5/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.9517 - loss: 0.1400 - val_accuracy: 0.9746 - val_loss: 0.0753
>>> TASK 4 COMPLETE. ALL EXERCISES FINISHED.
